In [ ]:
import random
import pickle

# Inner Code $=$ Reed-Muller($GF(q), m, d$)

In [249]:
from reed_muller import ReedMullerCode

In [262]:
inner_code = ReedMullerCode(2, 2, 4)
inner_code.length

16

In [4]:
with open("inner_code.obj", 'wb') as file:
    pickle.dump(inner_code, file)

In [264]:
inner_codeword = inner_code.random_codeword()
inner_codeword

[z2 + 1, 1, z2 + 1, 1, 0, 1, 0, 1, z2 + 1, 1, z2 + 1, 1, 0, 1, 0, 1]

### If this is a good codeword for the yet to be built Cayley graph since all symbols appear an even number of times. Store it in a file.

In [6]:
# with open("inner_codeword.obj", 'wb') as file:
#     pickle.dump(inner_codeword, file)

In [265]:
query_set = inner_code.build_query_set(3)
query_set

[12, 7, 10]

In [266]:
symbols = [inner_codeword[pos] for pos in query_set]
inner_code.reconstruct(3, symbols)

z2

# Cayley Graph & Codeword

### Create a Cayley graph of degree $|S|$ and $|\mathbb{SL}(2,F)|$ vertices. The symbol at position $i$ from the codeword will be assigned to the generator at position $i$ in $S$. Hence make sure that the same symbol is assigned to $s$ and $s^{-1}$.

In [13]:
F = GF(9)
group = SL(2,F)

paired_pos = [(0, 2), (1, 3), (4, 6), (5, 7), 
              (8, 10), (9, 11), (12, 14), (13, 15)]

used = set()
S = [None] * 16

def place(m):
    minv = m.inverse()
    minv.set_immutable()
    m.set_immutable()

    if m not in used and minv not in used:
        S[paired_pos[-1][0]] = m
        S[paired_pos[-1][1]] = minv

        paired_pos.pop()
        used.add(m)

for x in F:
    if x != 0:
        place(matrix(F, 2, 2, [[1, x],[0,1]]))
        place(matrix(F, 2, 2, [[1, 0],[x,1]]))

print(S)
    
graph = Graph(group.cayley_graph(generators=S))

print(graph.is_regular())
# print(len(graph.edges_incident(graph.vertices()[0])))
# print(len(graph.vertices()))
# graph.plot(edge_labels=True)

[[1 0]
[2 1], [1 2]
[0 1], [1 0]
[1 1], [1 1]
[0 1], [       1        0]
[2*z2 + 1        1], [       1 2*z2 + 1]
[       0        1], [     1      0]
[z2 + 2      1], [     1 z2 + 2]
[     0      1], [     1      0]
[z2 + 1      1], [     1 z2 + 1]
[     0      1], [       1        0]
[2*z2 + 2        1], [       1 2*z2 + 2]
[       0        1], [ 1  0]
[z2  1], [ 1 z2]
[ 0  1], [   1    0]
[2*z2    1], [   1 2*z2]
[   0    1]]
True


### Build it's double cover.

In [14]:
double_cover = BipartiteGraph(graph.bipartite_double())
# double_cover.plot(edge_labels=True)

### Label the edges with (left pos, right pos, global pos).

In [20]:
no_edges = double_cover.size()

for pos, e in enumerate(double_cover.edges()):
    e0 = e[0][0]
    e1 = e[1][0]
    
    if e[0][1] != 0:
        e0, e1 = e1, e0
    
    mat = e0.inverse() * e1
    if mat not in S:
        raise Exception()
            
    pos0 = S.index(mat)
    mat_inv = mat.inverse()
    
    for idx, m in enumerate(S):
        if idx != pos0 and mat_inv == m:
            pos1 = S.index(mat.inverse())
    
    double_cover.set_edge_label(e[0], e[1], (pos0, pos1, pos))

# double_cover.plot(edge_labels=True)

### Make sure the labelling is correct.

In [21]:
for e in double_cover.edges():
    if e[2] == None:
        raise Exception()
        
for v in double_cover.vertices():
    used = [False for _ in range(16)]
    
    for e in double_cover.edges_incident(v):    
        if used[e[2][v[1]]]:
            print(v)
            
            for e in double_cover.edges_incident(v):
                print(e)
                
            raise Exception()
            
        used[e[2][v[1]]] = True

### The vertices of the double cover are required to be labeled like: $(\cdot, \text{graph side})$

### Construct an outer codeword using the inner one, the double cover and the symbol assignments to edges of the graph.

In [81]:
outer_codeword = []
for e in loaded_double_cover.edges():
    outer_codeword.append(loaded_codeword[e[2][0]])

[z2 + 1,
 z2 + 1,
 0,
 z2 + 1,
 0,
 0,
 0,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2 + 1,
 0,
 z2 + 1,
 0,
 0,
 0,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2 + 1,
 z2 + 1,
 0,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2 + 1,
 z2 + 1,
 0,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 0,
 z2 + 1,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 0,
 z2 + 1,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2 + 1,
 0,
 z2 + 1,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2 + 1,
 0,
 z2 + 1,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 0,
 0,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 z2 + 1,
 z2,
 z2,
 0,
 z2,
 0,
 0,
 0,
 z2,
 

### Save the double cover and outer codeword.

In [82]:
with open("outer_codeword.obj", 'wb') as file:
    pickle.dump(outer_codeword, file)

In [25]:
# with open("double_cover.obj", 'wb') as file:
#     pickle.dump(double_cover, file)

### Load the double cover, inner codeword and outer codeword.

In [4]:
with open("double_cover.obj", 'rb') as file:
    loaded_double_cover = pickle.load(file)

In [27]:
print('degree: ', loaded_double_cover.degree()[0])
print('# vertices: ', len(loaded_double_cover.vertices()))
print('regular? ', loaded_double_cover.is_regular())

degree:  16
# vertices:  1440
regular?  True


In [109]:
with open("outer_codeword.obj", 'rb') as file:
    loaded_outer_codeword = pickle.load(file)

In [150]:
with open("inner_codeword.obj", 'rb') as file:
    loaded_inner_codeword = pickle.load(file)

# Locally Correctable Expander Code

In [6]:
from reed_muller import ReedMullerCode
# from lcec import *
from codeword_reader import CodewordReader

import pdb

In [7]:
inner_code = ReedMullerCode(2, 2, 4)

In [248]:
import math 

from sage.graphs.bipartite_graph import BipartiteGraph

from reconstructible_code import ReconstructibleCode

class LCECode():
    """Class implementing the localy correctable expander code."""

    class BGraph:
        class Edge:
            def __init__(self, pos):
                self.global_pos = pos[2]
                self.local_pos  = (pos[0], pos[1])
                self.vertices   = [None, None]

        class Vertex:
            def __init__(self, degree):
                self.edges = [None] * degree

        def __init__(self, double_cover, lcec):
            self.edges    = [None] * double_cover.size()
            self.vertices = [None] * (2  * len(self.edges) // lcec.dc_degree)

            idx = 0
            for vertex in double_cover.left:
                self.vertices[idx] = LCECode.BGraph.Vertex(lcec.dc_degree)

                for e in double_cover.edges_incident(vertex):
                    if self.edges[e[2][2]] != None:
                        raise Exception('Two different edges can not have the same global position.')
                    
                    if self.vertices[idx].edges[e[2][0]] != None:
                        raise Exception('Two different edges can not have the same local position.')

                    self.edges[e[2][2]] = LCECode.BGraph.Edge(e[2])
                    self.vertices[idx].edges[e[2][0]] = self.edges[e[2][2]]
                    self.edges[e[2][2]].vertices[0] = self.vertices[idx]

                idx += 1

            for vertex in double_cover.right:
                self.vertices[idx] = LCECode.BGraph.Vertex(lcec.dc_degree)

                for e in double_cover.edges_incident(vertex):
                    if self.vertices[idx].edges[e[2][1]] != None:
                        raise Exception('Two different edges can not have the same local position.')

                    self.vertices[idx].edges[e[2][1]] = self.edges[e[2][2]]
                    self.edges[e[2][2]].vertices[1] = self.vertices[idx]

                idx += 1


    class Tree:
        def __init__(self, edge, lcec):
            self.children   = []
            self.gedge      = edge
            self.lcec       = lcec
            self.symbol     = None

            if edge == None:
                raise ValueError('None edge detected.')
        

        def make_tree(self, side, depth):
            assert(depth >= 1)

            if depth == 1:
                return [self]

            leaves = []

            vertex = self.gedge.vertices[side]

            pos_to_be_reconstructed = self.gedge.local_pos[side]

            query_set = self.lcec.inner_code.build_query_set(pos_to_be_reconstructed)

            for query_pos in query_set:
                
                child = LCECode.Tree(vertex.edges[query_pos], self.lcec)
                self.add_child(child)

                leaves += child.make_tree(1 - side, depth - 1)
            
            return leaves


        def correct_lower_tree(self, side):
            for child in self.children:
                child.correct_lower_tree(1 - side)

            lcec = self.lcec
            self.path_dist  = dict()

            if not self.children:
                for symbol in lcec.alphabet:
                    self.path_dist[symbol] = int(symbol != self.symbol)
            else:
                if self.gedge.local_pos[side] not in lcec.inner_code_rec_results:
                    lcec.inner_code_rec_results[self.gedge.local_pos[side]] = dict()
                    
                    build_symbol_to_sets(lcec.inner_code,
                                self.gedge.local_pos[side], 
                                lcec.inner_code_rec_results[self.gedge.local_pos[side]])
                    
                symbol_to_sets = lcec.inner_code_rec_results[self.gedge.local_pos[side]]

                for reconstructed_symbol in lcec.alphabet:
                    min_path_dist = float('inf')

                    if reconstructed_symbol in symbol_to_sets:
                        for symbols in symbol_to_sets[reconstructed_symbol]:
                            max_path_dist = 0

                            for idx, symbol in enumerate(symbols):
                                max_path_dist = max(max_path_dist, 
                                                    self.children[idx].path_dist[symbol])
                            
                            min_path_dist = min(max_path_dist, min_path_dist)
                    
                    self.path_dist[reconstructed_symbol] = min_path_dist + 1

                min_path_dist = self.path_dist[lcec.alphabet[0]] + 1

                for reconstructed_symbol in lcec.alphabet:
                    if self.path_dist[reconstructed_symbol] < min_path_dist:
                        min_path_dist = self.path_dist[reconstructed_symbol]
                        self.symbol   = reconstructed_symbol


        def reconstruct_top_tree(self, side):
            if self.symbol != None:
                return
        
            symbols = []    
            for child in self.children:
                child.reconstruct_top_tree(1 - side)
                symbols.append(child.symbol)

#             breakpoint()
            self.symbol = self.lcec.inner_code.reconstruct(self.gedge.local_pos[side], symbols)
            

        def add_child(self, node):
            assert isinstance(node, LCECode.Tree)
            self.children.append(node)


        def add_positions_to_dict(self, pos_to_symbol):
            pos_to_symbol[self.gedge.global_pos] = None
            
            for child in self.children:
                child.add_positions_to_dict(pos_to_symbol)


        def fill_symbols(self, pos_to_symbol):
            self.symbol = pos_to_symbol[self.gedge.global_pos]

            for child in self.children:
                child.fill_symbols(pos_to_symbol)


    def __init__(self, inner_code : ReconstructibleCode, double_cover : BipartiteGraph):
        if not double_cover.is_regular():
            raise ValueError("the double cover needs to be regular.")
        
        self.dc_degree = double_cover.degree()[0]

        if self.dc_degree != inner_code.length:
            raise ValueError("the double cover's regularity degree is not equal to the length of the inner code (" + 
                                str(self.cd_degree) + " != " + str(inner_code.length) + ").")

        self.inner_code = inner_code
        self.alphabet   = inner_code.alphabet
        self.length     = 2 * double_cover.size()
        self.dc         = LCECode.BGraph(double_cover, self)

        self.inner_code_rec_results = dict()


    def correct(self, pos, codeword_reader):
        root, top_leaves, bottom_leaves = self.build_reconstruction_tree(pos)
        
        self.fill_bottom_trees_with_symbols(top_leaves, codeword_reader)

        for vertex in top_leaves:
            vertex.correct_lower_tree(1)

        root.reconstruct_top_tree(1)

        return root.symbol


    def build_reconstruction_tree(self, pos):
        depth1 = 1 #max(2, int(math.log(len(self.dc.vertices))) // self.dc_degree)
        #TODO: find the right value
        depth2 = 2 #depth1

        edge        = self.dc.edges[pos]
        root        = LCECode.Tree(edge, self)
        top_leaves  = root.make_tree(1, depth1)

        bottom_leaves = []
        for leaf in top_leaves:
            bottom_leaves += leaf.make_tree(1, depth2)

        return root, top_leaves, bottom_leaves


    def fill_bottom_trees_with_symbols(self, roots, codeword_reader):
        pos_to_symbol = dict()
        for vertex in roots:
            vertex.add_positions_to_dict(pos_to_symbol)
        
        codeword_reader.fill(pos_to_symbol)

        for vertex in roots:
            vertex.fill_symbols(pos_to_symbol)


def build_symbol_to_sets(reconstructible_code, pos, symbol_to_sets, symbols=[]):
    if len(symbols) == reconstructible_code.query_complexity:
        symbol = reconstructible_code.reconstruct(pos, symbols)
        
        if symbol not in symbol_to_sets:
            symbol_to_sets[symbol] = []
        
        symbol_to_sets[symbol].append(copy(symbols))
    else:
        for symbol in reconstructible_code.alphabet:
            symbols.append(symbol)
            build_symbol_to_sets(reconstructible_code, pos, symbol_to_sets, symbols)
            symbols.pop()

In [243]:
code = LCECode(inner_code, loaded_double_cover)

In [90]:
reader = CodewordReader(loaded_codeword)

In [151]:
loaded_inner_codeword

[z2, z2 + 1, z2, z2 + 1, 0, 0, 0, 0, z2, z2 + 1, z2, z2 + 1, 0, 0, 0, 0]

In [116]:
code.alphabet

[0, z2, z2 + 1, 1]

In [115]:
loaded_outer_codeword[3]

z2 + 1

In [247]:
code.correct(3, reader)

2 2
Reading pos:  137 from codeword. Symbol:  0
Reading pos:  90 from codeword. Symbol:  z2 + 1
Reading pos:  34 from codeword. Symbol:  0
Reading pos:  139 from codeword. Symbol:  z2
Reading pos:  138 from codeword. Symbol:  0
Reading pos:  132 from codeword. Symbol:  z2
Reading pos:  130 from codeword. Symbol:  0
Reading pos:  3 from codeword. Symbol:  z2 + 1
Reading pos:  131 from codeword. Symbol:  z2 + 1
Reading pos:  136 from codeword. Symbol:  0
Reading pos:  135 from codeword. Symbol:  z2


z2

In [217]:
query_set = inner_code.build_query_set(11)
query_set

[10, 12, 7]

In [220]:
print(loaded_inner_codeword[3])
symbols = [loaded_inner_codeword[pos] for pos in query_set]
inner_code.reconstruct(3, symbols)

z2 + 1


z2

In [199]:
print(code.dc.edges[3].local_pos[0])

for idx, e in enumerate(code.dc.edges[3].vertices[1].edges):
    print (idx, e.global_pos, loaded_outer_codeword[e.global_pos])

11
0 133 z2
1 128 z2 + 1
2 132 z2
3 90 z2 + 1
4 138 0
5 129 0
6 136 0
7 63 0
8 135 z2
9 3 z2 + 1
10 139 z2
11 131 z2 + 1
12 134 0
13 34 0
14 137 0
15 130 0


In [190]:
inner_code.reconstruct(3, [code.alphabet[2], code.alphabet[1], code.alphabet[3]])

0